In [2]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using MechanismGeometries
using CoordinateTransformations
using GeometryBasics

include("/home/hkolano/onr-dynamics-julia/simulate_with_ext_forces.jl")
println("Libraries imported.")

Libraries imported.


In [7]:
# model definition
# ------------------------------------------------------------------------
#                            MODEL DEFINITION
# ------------------------------------------------------------------------
vis = Visualizer()
render(vis)
# Define paths to the URDF files
src_dir = dirname(pathof(RigidBodyDynamics))
urdf_file_combo = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "URDFs", "alphaArmCombo.urdf")
mechanism_alpha = parse_urdf(urdf_file_combo; gravity = [0.0, 0.0, 0.0])

delete!(vis)

# Create visuals of the URDFs
visuals = URDFVisuals(urdf_file_combo)
mvis_alpha = MechanismVisualizer(mechanism_alpha, URDFVisuals(urdf_file_combo), vis[:alpha])

# Name the joints of the mechanisms
base_joint, shoulder_joint, elbow_joint, wrist_joint = joints(mechanism_alpha)
~, shoulder_body, upper_arm_body, elbow_body, wrist_body = bodies(mechanism_alpha)

shoulder_frame = default_frame(shoulder_body)
upper_arm_frame = default_frame(upper_arm_body)
elbow_frame = default_frame(elbow_body)
wrist_frame = default_frame(wrist_body)
base_frame = root_frame(mechanism_alpha)
# # last_link = bodies(mechanism_alpha)[end]
# # body_frame = default_frame(last_link)

frame_names_cob = ["shoulder_cob", "ua_cob", "elbow_cob", "wrist_cob"]
frame_names_com = ["shoulder_com", "ua_com", "elbow_com", "wrist_com"]
cob_vecs = [SVector{3, Float64}([-0.001, -0.003, .032]), SVector{3, Float64}([0.073, 0.0, -0.002]), SVector{3, Float64}([0.015, -0.012, -.003]), SVector{3, Float64}([0.0, 0.0, -.098])]
com_vecs = [SVector{3, Float64}([0.005, -.001, 0.016]), SVector{3, Float64}([0.073, 0.0, 0.0]), SVector{3, Float64}([0.017, -0.026, -0.002]), SVector{3, Float64}([0.0, 0.003, -.098])]
cob_frames = []
com_frames = []
for i in 1:4
    bod = bodies(mechanism_alpha)[i+1]
    frame_cob = CartesianFrame3D(frame_names_cob[i])
    frame_com = CartesianFrame3D(frame_names_com[i])
    cob_vec = cob_vecs[i]
    com_vec = com_vecs[i]
    cob_transform = Transform3D(frame_cob, default_frame(bod), cob_vec)
    com_transform = Transform3D(frame_com, default_frame(bod), com_vec)
    # setelement!(mvis_alpha, default_frame(bod))
    if !(is_fixed_to_body(bod, frame_cob))
        add_frame!(bod, cob_transform)
        push!(cob_frames, frame_cob)
        # setelement!(mvis_alpha, frame)    # visualizes COB frames in MeshCat
    end
    if !(is_fixed_to_body(bod, frame_com))
        add_frame!(bod, com_transform)
        push!(com_frames, frame_com)
        # setelement!(mvis_alpha, frame_com)    # visualizes COM frames in MeshCat
    end
end

println("URDFs parsed. Visualizers ready. \n")

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8706
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


URDFs parsed. Visualizers ready. 



In [8]:
# buoyancy force setup
# ------------------------------------------------------------------------
#                           BUOYANCY SETUP
# ------------------------------------------------------------------------
# f = rho * g * V
# f = 997 (kg/m^3) * 9.81 (m/s^2) * V_in_L *.001 (m^3) = kg m / s^2
# One time setup of buoyancy forces
volumes = [.018, .203, .025, .155] # shoulder, ua, elbow, wrist 
buoy_force_mags = volumes * 997 * 9.81 * .001
buoy_lin_forces = []
for mag in buoy_force_mags
    lin_force = FreeVector3D(base_frame, [0.0, 0.0, mag])
    push!(buoy_lin_forces, lin_force)
end

masses = [.194, .429, .115, .333]
grav_forces = masses*9.81
grav_lin_forces = []
for f_g in grav_forces
    lin_force = FreeVector3D(base_frame, [0.0, 0.0, -f_g])
    push!(grav_lin_forces, lin_force)
end

# println(spatial_inertia(shoulder_body))
# println(spatial_inertia(upper_arm_body))
# println(spatial_inertia(elbow_body))
# println(spatial_inertia(wrist_body))
# println(mass(mechanism_alpha))

In [9]:
# Hydrodynamics calculator function
# ------------------------------------------------------------------------
#                HYDRODYNAMICS (Grav & Buoy) CALCULATOR
# ------------------------------------------------------------------------
function hydro_calc!(hydro_wrenches::Dict{BodyID, Wrench{Float64}}, t, state_alpha::MechanismState)
    buoy_wrenches = []
    grav_wrenches = []
    # Iterate through buoyancy for each body
    for i in 1:4
        # Get the body (will have to change this index when add more joints)
        bod = bodies(mechanism_alpha)[i+1]
        # Get default frame of te body
        body_default_frame = default_frame(bod)
        # Get transform between the defualt frame and the center of buoyancy
        def_to_cob = fixed_transform(bod, body_default_frame, cob_frames[i])
        def_to_com = fixed_transform(bod, body_default_frame, com_frames[i])
        # Transform buoyancy force vector to the body's default frame (rotation only)
        buoy_force_trans = transform(state_alpha, buoy_lin_forces[i], body_default_frame)
        grav_force_trans = transform(state_alpha, grav_lin_forces[i], body_default_frame)
        # Make the wrench: the buoyancy force through a point, the center of buoyancy.
        buoy_wrench = Wrench(Point3D(body_default_frame, translation(inv(def_to_cob))), buoy_force_trans)
        grav_wrench = Wrench(Point3D(body_default_frame, translation(inv(def_to_com))), grav_force_trans)
        # Add wrench to buoy_wrenches
        push!(buoy_wrenches, buoy_wrench)
        push!(grav_wrenches, grav_wrench)
        # Add the buoyancy wrench and grav wrench together
        wrench = buoy_wrench + grav_wrench
        # Transform the wrench to the root frame and assign it to the body
        hydro_wrenches[BodyID(bod)] = transform(state_alpha, wrench, root_frame(mechanism_alpha))
    end
end;


In [10]:
# Controller function
# ------------------------------------------------------------------------
#                              CONTROLLER
# ------------------------------------------------------------------------

function simple_control!(torques::AbstractVector, t, state_alpha::MechanismState, hydro_calc!)
    # Get buoyancy buoyancy forces 
    hydro_calc!(hydro_wrenches, t, state_alpha) 
    # Calculate inverse dynamics of alpha arm (including buoyancy)
    tau = inverse_dynamics(state_alpha, des_acc, hydro_wrenches)

    println("Control Tau:")
    println(tau)

    # Set torques accordingly (controller)
    torques[velocity_range(state_alpha, base_joint)] .= tau[base_joint][1]
    torques[velocity_range(state_alpha, shoulder_joint)] .= tau[shoulder_joint][1]
    torques[velocity_range(state_alpha, elbow_joint)] .= tau[elbow_joint][1]
    torques[velocity_range(state_alpha, wrist_joint)] .= tau[wrist_joint][1]
    # torques[velocity_rangege(state_alpha, base_joint)] .= 0.01
    # torques[velocity_range(state_alpha, shoulder_joint)] .= 0.01
    # torques[velocity_range(state_alpha, elbow_joint)] .= 0.01
    # torques[velocity_ran(state_alpha, wrist_joint)] .= 0.01
end;

In [12]:
# initial conditions
# ------------------------------------------------------------------------
#                          INITIAL CONDITIONS
# ------------------------------------------------------------------------
# Initialize state of alpha arm
state_alpha = MechanismState(mechanism_alpha)
zero!(state_alpha)
# set_configuration!(state, base_joint, 0.0)
# set_configuration!(state, shoulder_joint, -.27079)
# set_configuration!(state, elbow_joint, 3.1415/2+1.3)
# set_configuration!(state, wrist_joint, 0.5)

# Define desired accelerations
des_acc = similar(velocity(state_alpha))
des_acc[base_joint][1] = 0.0001
des_acc[shoulder_joint][1] = 0.
des_acc[elbow_joint][1] = 0.
des_acc[wrist_joint][1] = 0.

0.0

In [14]:
# ------------------------------------------------------------------------
#                              SIMULATION
# ------------------------------------------------------------------------
final_time = 1.0
ts, qs, vs = simulate_with_ext_forces(state_alpha, final_time, hydro_calc!, simple_control!; Δt = 5e-3)
# ts, qs, vs = simulate(state_alpha, final_time, simple_control!; Δt = 1e-2)

println("Simulation finished.")

------ NEW SIM STATE -----
Control Tau:
[2.1573244969520816e-6, 0.3391544392769082, 0.1979696112702415, -0.008600275846047646]


LoadError: MethodError: no method matching hydro_calc!(::Dict{BodyID, Wrench{Float64}}, ::Float64, ::MechanismState{Float64, Float64, Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, Revolute{Float64}}}}, 1}}, ::SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}})
[0mClosest candidates are:
[0m  hydro_calc!(::Dict{BodyID, Wrench{Float64}}, ::Any, ::MechanismState) at In[9]:5

In [ ]:
# ------------------------------------------------------------------------
#                             VISUALIZATION
# ------------------------------------------------------------------------
# vis = Visualizer()
# open(vis)

# delete!(vis)
# mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf_file))
# set_configuration!(mvis, [0.0, 0.0])
# open(mvis)
MeshCatMechanisms.animate(mvis_alpha, ts, qs; realtimerate = 1.)
print(last(qs))

# animation = Animation(mvis, ts, qs)
# setanimation!(mvis, animation)

println("\n done.")

With added mass: 
[0.00547, 0.69718, 0.93936, 1.07620]

without added mass:
[0.005475504, 0.697184625, 0.93936468, 1.0762014]